# Fine-tuning on the reference dataset
In this notebook, we'll demonstrate fine-tuning a pre-trained CNN on the 30-isolate classification task shown in Figure 2. In this example, fine-tuning serves to update the CNN to new measurement parameters. This code illustrates the procedure described in the `CNN architecture & training details` section in the Methods. Note that for speed and clarity, this demo only trains on a single randomly selected train and validation split.

这段Markdown文本主要目的是介绍一个Jupyter Notebook的内容，该Notebook演示了如何对一个预训练的卷积神经网络（CNN）进行微调，以应用于一个30个菌株的分类任务。微调的目的是为了使CNN能够适应新的测量参数。代码遵循了方法部分中关于CNN架构和训练流程的具体描述。需要注意的是，为了演示的简洁性，仅使用了一个随机的数据集划分来进行训练和验证。

## Loading data
The first step is to load the fine-tuning dataset.

导入数据
第一步是加载微调数据集。


In [1]:
from time import time
t00 = time()
import numpy as np

这段代码的主要功能是初始化程序的运行时间计时，并导入numpy库以便于后续的数据处理和科学计算。通过记录开始时间t00，用户可以在程序的其他部分使用这个变量来计算程序运行的总时间。

In [2]:
X_fn = './data/X_finetune.npy'
y_fn = './data/y_finetune.npy'
X = np.load(X_fn)
y = np.load(y_fn)
print(X.shape, y.shape)

(3000, 1000) (3000,)


前两行是定义文件路径，这里定义了两个变量X_fn和y_fn，分别存储特征数据（X）和标签数据（y）的文件路径。这两个文件是以.npy格式存储的NumPy数组文件。第三、四行使用np.load()函数从指定的路径加载数据文件到内存中。X和y分别存储了特征数据和标签数据。np.load()函数能够读取.npy格式的文件，并将其转换为NumPy数组，便于后续的数据处理和模型训练。第五行是打印数据形状，这一步可以帮助用户快速检查数据的维度是否符合预期，确保数据加载正确无误。本实验中，X是一个形状为(3000, 1000)的数组，这意味着有3000个样本，每个样本有1000个特征；而y的形状为(1000)，则表示有1000个对应的标签。


## Loading pre-trained CNN
Now we set up a ResNet CNN and load weights that we previously trained for the 30-isolate task using the full training dataset. 

这段代码片段描述了接下来要执行的操作步骤，即加载一个预训练的卷积神经网络（CNN）。具体来说，这段代码的目的是设置一个ResNet架构的CNN模型，并加载之前为处理30个特定菌株分类任务所训练的权重。

In [3]:
from resnet import ResNet
import os
import torch

（1）from resnet import ResNet: 这行代码是从名为 resnet 的模块中导入 ResNet 类。ResNet 是一种深度学习中的卷积神经网络模型，特别适用于图像分类任务。通过导入 ResNet 类，用户可以在当前的项目中使用这种模型进行细菌图像的识别或分类。

（2）import os: 这行代码导入了 Python 的 os 库。os 库提供了与操作系统交互的功能，例如文件路径操作、文件和目录的管理等。在后续的代码中，用户可能会使用 os 库来处理文件路径，读取或写入文件等。

（3）import torch: 这行代码导入了 torch 库，这是 PyTorch 深度学习框架的核心库。PyTorch 是一个广泛使用的深度学习库，提供了强大的张量计算（带有 GPU 加速的功能）、自动微分和构建深度神经网络的功能。通过导入 torch，用户可以在项目中进行深度学习模型的训练、评估和推理。

总结：这段代码的主要功能是为后续的细菌图像识别或分类任务导入必要的库和模型。具体来说，它导入了 ResNet 类用于构建卷积神经网络模型，导入了 os 库用于处理文件和目录相关的操作，以及导入了 torch 库作为深度学习框架的核心，支持张量运算和模型构建。这些导入为用户进一步在 1_reference_finetuning.ipynb 文件中进行模型的微调、训练和评估奠定了基础。

In [4]:
# CNN parameters
layers = 6
hidden_size = 100
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1000
in_channels = 64
n_classes = 30
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()

（1）定义CNN网络参数：
layers = 6: 设置卷积神经网络（CNN）的层数为6层。
hidden_size = 100: 每个隐藏层中神经元的数量设为100。
block_size = 2: 每个卷积块的大小设为2。
hidden_sizes = [hidden_size] * layers: 创建一个包含6个元素的列表，每个元素都是100，表示每层隐藏层的神经元数目。
num_blocks = [block_size] * layers: 创建一个包含6个元素的列表，每个元素都是2，表示每层卷积块的大小。

（2）定义输入和输出参数：
input_dim = 1000: 输入数据的维度为1000，这通常意味着输入数据的特征数量是1000。
in_channels = 64: 输入数据的通道数为64，这通常用于图像数据，表示图像的特征图数量。
n_classes = 30: 输出类别数量为30，这表示该CNN模型被设计用来分类30种不同的类别。

（3）设置CUDA设备：
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0): 设置环境变量，使得程序只使用GPU设备编号为0的GPU进行计算。
cuda = torch.cuda.is_available(): 检查当前环境中是否可用CUDA设备，即是否能够使用GPU进行计算，并将结果存储在变量cuda中。

总结：这段代码的主要功能是设置卷积神经网络（CNN）的参数和环境配置。具体来说，它定义了CNN网络的结构参数，如层数、每层的隐藏神经元数量、卷积块大小、输入数据的维度和通道数以及输出的类别数量。同时，它还配置了GPU环境，优先使用设备编号为0的GPU来加速计算，前提是当前环境中支持CUDA

In [5]:
# Load trained weights for demo
cnn = ResNet(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)
if cuda: cnn.cuda()
cnn.load_state_dict(torch.load(
    './pretrained_model.ckpt', map_location=lambda storage, loc: storage))

C:\Users\31458\AppData\Local\Temp\ipykernel_6536\1257292563.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn.load_state_dict(torch.load(


<All keys matched successfully>

（1）第一行：创建ResNet实例：这行代码创建了一个ResNet实例。ResNet是一种深度卷积神经网络架构，常用于图像分类任务。在创建实例时，需要指定网络的几个关键参数：
hidden_sizes：网络中每个残差块（residual block）中隐藏层的大小。
num_blocks：每个残差层（residual layer）中包含的残差块的数量。
input_dim：输入图像的空间维度（高度和宽度）。
in_channels：输入图像的通道数（例如，灰度图像为1，彩色RGB图像为3）。
n_classes：输出的类别数，即网络需要分类的类别总数。

（2）第二行：将模型移动到GPU（如果可用），这段代码检查变量cuda，如果其值为True，则将模型移动到GPU（图形处理器）上进行加速计算。

（3）第三行：这行代码加载了一个预训练模型的权重文件。具体来说：
torch.load：用于从指定的文件路径加载模型的权重。
'./pretrained_model.ckpt'：预训练模型权重文件的路径，通常是一个.ckpt或.pth文件。
map_location=lambda storage, loc: storage：这个参数用于指定加载模型时的位置映射。在这里，它的作用是将模型加载到当前设备上，无论这个设备是CPU还是GPU。这使得代码更具灵活性，不需要显式指定设备。

总结：这段代码的主要功能是加载一个预训练的ResNet模型的权重，准备在后续的演示或微调任务中使用该模型。
下面的两行说明是将对应的参数设置进去

## Fine-tuning
Now we can fine-tune the pre-trained CNN on the new fine-tuning dataset. In the experiments reported in the paper, we fine-tune across 5 randomly selected train and validation splits, but here we show just one split for clarity. We also only train for one epoch here in the interest of time. To train the CNN to convergence, we recommend setting the number of epochs to ~30.

这段Markdown文本的主要目的是解释和说明在细菌分类任务中，如何对预训练的卷积神经网络进行微调。它强调了微调的重要性，并指出了在实际操作中如何进行多轮次的随机划分来评估模型的稳定性和推荐的训练轮次（epoch）数量，以达到模型的最佳性能。虽然这段文本没有实际的代码，但它为后面具体的微调操作提供了理论指导和建议。

In [6]:
from datasets import spectral_dataloader
from training import run_epoch
from torch import optim

（1）from datasets import spectral_dataloader：
这行代码导入了一个名为 spectral_dataloader 的函数或类，该函数或类来自于 datasets 模块。根据名称可以推测，spectral_dataloader 可能是一个专门用来加载光谱数据的数据加载器。它可能负责从文件系统或其他数据源中读取光谱数据，并将其格式化为可以在模型中使用的张量或其他数据结构。

（2）from training import run_epoch：
这行代码从 training 模块中导入了一个名为 run_epoch 的函数。根据名称可以推测，run_epoch 函数可能是用于执行一个训练周期（训练的一个完整数据集迭代）的。它可能包含了前向传播、损失计算、反向传播以及参数更新等典型的训练步骤。

（3）from torch import optim：
这行代码从 torch 模块中导入了 optim 子模块。optim 包含了多种常用的优化算法，如随机梯度下降（SGD）、Adam 等。在深度学习模型训练中，通常使用这些优化算法来调整模型参数以最小化损失函数。

总结：这段代码的主要功能是为后续的光谱数据分析和模型训练做准备。它导入了用于加载光谱数据的数据加载器、执行训练周期的函数，以及用于优化模型参数的工具包。

### Train/val split
We split the fine-tuning dataset into train and validation sets. We randomly sample 10% of the dataset to use as a validation set.

这段代码的主要功能是描述如何将给定的微调数据集划分为训练集和验证集，以确保模型能够有效地学习并评估其性能。尽管这段文本本身不是代码，但它提供了对后续数据处理步骤的重要说明。

In [7]:
p_val = 0.1
n_val = int(3000 * p_val)
idx_tr = list(range(3000))
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

p_val = 0.1：定义了一个变量 p_val，表示验证集占整个数据集的比例为10%。

n_val = int(3000 * p_val)：计算验证集的数据量。这里假设数据集的总大小为3000，因此验证集的数据量 n_val 为300（即 3000 * 0.1 并取整）。

idx_tr = list(range(3000))：创建一个包含0到2999的整数列表 idx_tr，这个列表可以看作是数据集的索引。

np.random.shuffle(idx_tr)：使用NumPy库中的 random.shuffle 函数对 idx_tr 列表进行随机打乱，以确保训练集和验证集的选择是随机和无偏的。

idx_val = idx_tr[:n_val]：从随机打乱后的列表 idx_tr 中取出前 n_val 个索引，这些索引将用于验证集。

idx_tr = idx_tr[n_val:]：将 idx_tr 列表剩余的索引部分赋值给 idx_tr，这部分索引将用于训练集。

总结：这段代码的主要功能是将一个包含3000个样本的数据集，按照10%的比例随机划分成验证集和训练集，并分别存储为 idx_val 和 idx_tr 列表，其中这些列表包含了对应的数据索引。

In [8]:
# Fine-tune CNN
epochs = 1 # Change this number to ~30 for full training
batch_size = 10
t0 = time()
# Set up Adam optimizer
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Set up dataloaders
dl_tr = spectral_dataloader(X, y, idxs=idx_tr,
    batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X, y, idxs=idx_val,
    batch_size=batch_size, shuffle=False)
# Fine-tune CNN for first fold
best_val = 0
no_improvement = 0
max_no_improvement = 5
print('Starting fine-tuning!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.2f}s'.format(epoch+1, time()-t0))
    # Train
    acc_tr, loss_tr = run_epoch(epoch, cnn, dl_tr, cuda,
        training=True, optimizer=optimizer)
    print('  Train acc: {:0.2f}'.format(acc_tr))
    # Val
    acc_val, loss_val = run_epoch(epoch, cnn, dl_val, cuda,
        training=False, optimizer=optimizer)
    print('  Val acc  : {:0.2f}'.format(acc_val))
    # Check performance for early stopping
    if acc_val > best_val or epoch == 0:
        best_val = acc_val
        no_improvement = 0
    else:
        no_improvement += 1
    if no_improvement >= max_no_improvement:
        print('Finished after {} epochs!'.format(epoch+1))
        break

print('\n This demo was completed in: {:0.2f}s'.format(time()-t00))

Starting fine-tuning!
 Epoch 1: 0.00s
  Train acc: 77.89
  Val acc  : 86.67

 This demo was completed in: 114.98s


（1）Fine-tune CNN：设置训练轮数和批次大小，这里定义了训练的轮数（epochs），每次训练使用的数据量（batch_size），以及开始训练的时间点t0。注释提示用户可以将轮数增加到约30来完成完整的训练。

（2）Set up Adam optimizer：设置优化器，使用Adam优化器来更新CNN的参数。学习率（lr）设置为0.001，betas参数用于控制Adam优化器的两个动量参数，分别是0.5和0.999。

（3）Set up dataloaders：设置数据加载器，使用自定义的spectral_dataloader函数来创建训练数据加载器dl_tr和验证数据加载器dl_val。训练数据加载器会对数据进行随机打乱（shuffle=True），而验证数据加载器则不会（shuffle=False）。

（4）Fine-tune CNN for first fold：开始微调过程，初始化了两个变量：best_val用于记录验证集上的最佳准确率，no_improvement用于记录连续多少轮验证准确率没有提高。max_no_improvement设为5，意味着如果验证集上的准确率连续5轮没有提高，程序将提前终止训练。

（5）进行微调训练循环：每次循环代表一轮训练。对于每一轮，首先打印出当前的轮数和自训练开始以来所消耗的时间。然后调用run_epoch函数分别进行训练和验证过程。在训练过程中，training=True，而在验证过程中，training=False。训练和验证过程都会返回当前准确率和损失值，这些信息会被打印出来。接着检查验证集上的准确率，如果比之前保存的最佳准确率要高或者这是第一轮训练，则更新best_val并将no_improvement归零。如果验证集上的准确率没有提高，则no_improvement加一。如果no_improvement达到或超过max_no_improvement（5轮），则打印一条消息后跳出训练循环，提前终止训练。

（6）输出训练完成时间：本实验完成总时间为114.98s

The accuracies seen here are not representative of the accuracies achieved when training on the full dataset until convergence. To do this, increase the number of epoches. This code demonstrates how a pre-trained CNN can be fine-tuned and evaluated using randomly selected train/validation splits.

微调预训练CNN：利用随机选取的训练数据对一个已经训练好的CNN模型进行微调。

评估模型性能：在微调完成后，使用随机选取的验证数据来评估模型的性能。

提示改进：提醒用户通过增加训练轮数来获得更接近收敛的模型性能。